In [7]:
from scipy import io as sio
from sklearn.preprocessing import Imputer
import numpy as np

data_PFC = sio.loadmat("../data/GSE44772_PFC_refined.mat")
data_VC = sio.loadmat("../data/GSE44772_VC_refined.mat")
data_CR = sio.loadmat("../data/GSE44772_CR_refined.mat")

def setRegion(region):
    if region == 'PFC':
        ourdata = data_PFC
    elif region == 'VC':
        ourdata = data_VC
    elif region == 'CR':
        ourdata = data_CR
    elif region == 'All':
        ourdata = data_All
        
    inputX = ourdata['X']
    # Inpute using median
    imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
    imp.fit(inputX)
    inputX = imp.transform(inputX)
    
    inputY = ourdata['y'][0,:]
    
    return inputX, inputY

In [5]:
inputX_PFC, inputY = setRegion('PFC')
inputX_VC, inputY = setRegion('VC')
inputX_CR, inputY = setRegion('CR')

In [8]:
from sklearn.mixture import GMM

gmm = GMM(n_components=2)

def gmmDiscretize(x):
    counter[0] += 1
    if counter[0] % 1000 == 0:
        print counter
    return gmm.fit_predict(np.expand_dims(x,axis=1))

counter = [0]

newX_PFC_gmm = np.apply_along_axis(gmmDiscretize, 0, inputX_PFC)
newX_VC_gmm = np.apply_along_axis(gmmDiscretize, 0, inputX_VC)
newX_CR_gmm = np.apply_along_axis(gmmDiscretize, 0, inputX_CR)

[1000]
[2000]
[3000]
[4000]
[5000]
[6000]
[7000]
[8000]
[9000]
[10000]
[11000]
[12000]
[13000]
[14000]
[15000]
[16000]
[17000]
[18000]
[19000]
[20000]
[21000]
[22000]
[23000]
[24000]
[25000]
[26000]
[27000]
[28000]
[29000]
[30000]
[31000]
[32000]
[33000]
[34000]
[35000]
[36000]
[37000]
[38000]
[39000]
[40000]
[41000]
[42000]
[43000]
[44000]
[45000]
[46000]
[47000]
[48000]
[49000]
[50000]
[51000]
[52000]
[53000]
[54000]


In [13]:
from sklearn.mixture import GMM

gmm = GMM(n_components=2)

def medianDiscretize(x):
    counter[0] += 1
    if counter[0] % 1000 == 0:
        print counter
    median = np.median(x)
    x_new = np.copy(x)
    x_new[x<median] = 0
    x_new[x>=median] = 1
    return x_new

counter = [0]

newX_PFC_median = np.apply_along_axis(medianDiscretize, 0, inputX_PFC)
newX_VC_median = np.apply_along_axis(medianDiscretize, 0, inputX_VC)
newX_CR_median = np.apply_along_axis(medianDiscretize, 0, inputX_CR)

[1000]
[2000]
[3000]
[4000]
[5000]
[6000]
[7000]
[8000]
[9000]
[10000]
[11000]
[12000]
[13000]
[14000]
[15000]
[16000]
[17000]
[18000]
[19000]
[20000]
[21000]
[22000]
[23000]
[24000]
[25000]
[26000]
[27000]
[28000]
[29000]
[30000]
[31000]
[32000]
[33000]
[34000]
[35000]
[36000]
[37000]
[38000]
[39000]
[40000]
[41000]
[42000]
[43000]
[44000]
[45000]
[46000]
[47000]
[48000]
[49000]
[50000]
[51000]
[52000]
[53000]
[54000]


In [16]:
igs_PFC_median = getIGS(newX_PFC_median)
igs_VC_median = getIGS(newX_VC_median)
igs_CR_median = getIGS(newX_CR_median)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000


In [15]:
igs_PFC_gmm = getIGS(newX_PFC_gmm)
igs_VC_gmm = getIGS(newX_VC_gmm)
igs_CR_gmm = getIGS(newX_CR_gmm)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000


In [19]:
sio.savemat("../data/PFC_IG_refined.mat", {"gmm":{"x":newX_PFC_gmm, "ig":igs_PFC_gmm}, "median":{"x":newX_PFC_median,"ig":igs_PFC_median}})
sio.savemat("../data/CR_IG_refined.mat", {"gmm":{"x":newX_CR_gmm, "ig":igs_CR_gmm}, "median":{"x":newX_CR_median,"ig":igs_CR_median}})
sio.savemat("../data/VC_IG_refined.mat", {"gmm":{"x":newX_VC_gmm, "ig":igs_VC_gmm}, "median":{"x":newX_VC_median,"ig":igs_VC_median}})

In [9]:
from scipy.stats import entropy

def getIGS(newX):
    n_sample, n_feature = newX.shape[0], newX.shape[1]
    igs = []
    ori_entropy = entropy([len(inputY[inputY==0]),len(inputY[inputY==1])])

    for i in xrange(n_feature):
        if i % 1000 == 0:
            print i

        zero = inputY[newX[:,i]==0]
        one = inputY[newX[:,i]==1]

        if len(zero) != 0:
            entropy_zero = entropy([len(zero[zero==0]),len(zero[zero==1])])
        else:
            entropy_zero = 0

        if len(one) != 0:
            entropy_one = entropy([len(one[one==0]),len(one[one==1])])
        else:
            entropy_one = 0

        igs.append(ori_entropy-(len(zero)*entropy_zero*1.0/n_sample)-(len(one)*entropy_one*1.0/n_sample))
    
    return igs

In [60]:
x = newX[i,:]==04

In [105]:
if np.inf == 10:
    print 1

In [183]:
sio.savemat("PFC_IG.mat", {"binaryX":newX, "IG":igs_PFC})

In [182]:
igs_PFC_ = getIGS(newX)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
